In [1]:
import requests
import re
from bs4 import BeautifulSoup
import sqlite3


In [ ]:
def save_data(data:dict, database:sqlite3):
    insert = []
    cols = [
        'product_id', 'brand', 'weight', 'cpu', 'cpu_series', 'cpu_model', 'cpu_cache', 'freq',
        'freq_bound', 'gpu', 'gpu_model', 'gpu_size', 'HDD', 'SSD', 'screen_size', 'screen_acc',
        'port_vga', 'port_hdmi', 'port_thunderbolt', 'port_serial', 'usb3', 'usb2', 'usb4', 'usbc',
        'battery', 'os','ram','ram_kind', 'price'
    ]

    cols_default = [
        'product_id', 'brand', '-1', 'cpu', 'cpu_series', 'cpu_model', '-1', '-1',
        '-1', 'gpu', 'gpu_model', '-1', '0', '0', 'screen_size', 'screen_acc',
        '0', '0', '0', '0', '0', '0', '0', '0',
        '0', 'os','-1','unknown','0'
    ]
 
    for(col,default) in zip(cols, cols_default):
        try:
            if col in data: insert.append(data[col])
            else: insert.append(default)
        except: pass
    database.execute("INSERT INTO laptops VALUES('%s','%s','%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')"%tuple(insert))
    database.commit()

In [ ]:
def get_price(soup):
    pr = soup.find('div', attrs={'class': ['c-product__seller-price-pure','js-price-value']})
    num_parser = {
        '۱' : '1',
        '۲' : '2',
        '۳' : '3',
        '۴' : '4',
        '۵' : '5',
        '۶' : '6',
        '۷' : '7',
        '۸' : '8',
        '۹' : '9',
        '۰' : '0',
        ',' : '',
    }
    new_price = pr.text
    for p in num_parser:
        new_price = new_price.replace(p, num_parser[p])
    new_price = re.sub(r'\s+', '', new_price)
    return new_price

In [ ]:
def get_brand(soup):
    s = soup.find('span',attrs={'class': ['c-product__title-en']})
    try:
        text = re.search(r'[A-Z,a-z]+',s.text)
        return text.group(0)
    except: return 'unknown'

In [ ]:
def clean_v(value:str):
    changes = {
        'بدون حافظه‌ی گرافیکی مجزا':'0',
        'گیگاهرتز و بیشتر':'',
        'گیگاهرتز':'',
        'یک' : '1',
        'مگابایت' : '000000',
        'گیگابایت' : '000000000',
        'ترابایت' : '000000000000',
        'گیگ': '000000000',
        'GB' : '000000000',
        'MB' : '000000',
        'TB' : '000000000000',
        'میلی': '',
        'اینچ': '',
        'کیلوگرم': '',
        'تا': '_',
        'سلولی':'',
        'عدد':'',
        'up to':'_',
        ' گیگ': '000000000',
        'هارد دیسک':'HDD',
        '‌‌':'',#nim fasele
        ' ': '',
        'GHz':'',
    }
    for change in changes:
        value = value.replace(change, changes[change])
    return value

In [ ]:
def clean_k(key:str):
    changes = {
        ' ': '',
    }
    for change in changes:
        key = key.replace(change, changes[change])
    return key

In [ ]:
def storage(value):
    try:
        t = re.findall(r'[0-9]+',value)
        try: v1 = t[0]
        except : v1 = "0"
        try: v2 = t[1]
        except: v2 = "0"
        return v1, v2
    except: 
        return '0','0'

In [ ]:
def extract_product_detail(product, database, list_of_products):
    check = list_of_products.execute("SELECT * FROM products WHERE product = '%s'"%product).fetchone()
    if check[1] == 1: return
    else: print(check)
    url = 'https://www.digikala.com/product/{}/'.format(product)
    page = requests.get(url)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    detail_keys = soup.find_all('div',attrs={'class':['c-params__list-key']})
    deatil_list = soup.find_all('div', attrs={'class':['c-params__list-value']})    

    details = dict()

    replace_v = {
        'دارد':'1',
        'ندارد':'0',
        'بله':'1',
        'خیر':'0',
        'بدونسیستم':'0',
        'بدونحافظه':'0',
        'بدونویندوز': '0',
        'بدونسیستم‌عامل':'0',

    }

    replace_k = {
        'وزن' : 'weight',
        'نوعحافظهRAM':'ram_kind',
        'حافظهCache' : 'cpu_cache',
        'ظرفیتحافظهداخلی' : 'storage',
        'سازندهپردازندهگرافیکی' : 'gpu',
        'سازندهپردازنده' : 'cpu',
        'سریپردازنده' : 'cpu_series',
        'مدلپردازنده' : 'cpu_model',
        'ظرفیتحافظهRAM' : 'ram',
        'محدودهسرعتپردازنده' : 'freq_bound',
        'فرکانسپردازنده':'freq',
        'حافظهاختصاصیپردازندهگرافیکی': 'gpu_size',
        'مدلپردازندهگرافیکی' : 'gpu_model',
        'اندازهصفحهنمایش' : 'screen_size',
        'دقتصفحهنمایش' : 'screen_acc',
        'صفحهنمایشلمسی' : 'screen_touch',
        'صفحهنمایشمات' : 'screen_matt',
        'وبکم' : 'webcam',
        'توضیحاتوبکم' : 'webcam_quality',
        'حسگراثرانگشت' : 'fingerprint',
        'شبکهبیسیمWi' : 'wifi',
        'پورتشبکهEthernet' : 'port_ethernet',
        'پورتVGA' : 'port_vga',
        'پورتHDMI' : 'port_hdmi',
        'پورتThunderbolt' : 'port_thunderbolt',
        'تعدادپورتUSB3' : 'usb3',
        'تعدادپورتUSB2' : 'usb2',
        'پورتSerial': 'port_serial',
        'پورتUSBType' : 'usbc',
        'پورتUSB4' : 'usb4',
        'نوعباتری' : 'battery_kind',
        'سیستمعامل': 'os'

    }
    special = ['مشخصاتحافظهداخلی', 'نوعحافظهداخلی']

    for (key, value) in zip(detail_keys, deatil_list):
        details['brand'] = get_brand(soup).lower()

        k = re.search(r'\s*[A-Z,a-z,0-9,پچجحخهعغفقثصضشسیبلاتنمکگوئدذرزطظژؤإأءًٌٍَُِّ,(,),-,‌, ]+',key.text)
        try:
            k = re.sub(r'\s+', ' ', k.group(0))
        except:
            continue
        k = clean_k(k)
        if k in special:
            v = re.search(r'\s*[A-Z,a-z,0-9,پچجحخهعغفقثصضشسیبلاتنمکگوئدذرزطظژؤإأءًٌٍَُِّ,(,),-,‌, ]+',value.text)
            try:
                v = re.sub(r'\s+', ' ', v.group(0))
            except:
                continue
            v = clean_v(v)
            if k == 'مشخصاتحافظهداخلی':
                
                if v.find('SSD') >= 0 and v.find('HDD') >= 0:
                    v1, v2 = storage(v)
                    details["SSD"], details["HDD"] = v2, v1

                elif v.find('SSD') >= 0 and v.find('بههمراه') >= 0:
                    try:details['storage'] = re.search(r'[0-9]+',details['storage']).group(0)
                    except: pass
                    try :details['HDD'] = details['storage']
                    except: pass
                    tp = storage(v)
                    details['SSD'] = tp[0] 
            if k == 'نوعحافظهداخلی':
                if v == 'هارددیسک': v = 'HDD'
                if v == 'SSD' or v == 'HDD':
                    details[v] = details['storage']
                    try: details.pop('storage')
                    except : pass
        else:  
            if k in replace_k: k = replace_k[k]
            else: continue
            v = re.search(r'\s*[A-Z,a-z,0-9,پچجحخهعغفقثصضشسیبلاتنمکگوئدذرزطظژؤإأءًٌٍَُِّ,(,),-,‌,., ]+',value.text)
            v = re.sub(r'\s+', ' ', v.group(0))
            v = clean_v(v)
    
            if v in replace_v: v = replace_v[v]
            details[k] = v
        try:
            details['price'] = get_price(soup)
        except:
            details['price'] = 0
        details['product_id'] = product
        
    if 'HDD' not in details:details['HDD'] = '0'
    if 'SSD' not in details:details['SSD'] = '0'
    if details['HDD'] == '0' and  details['SSD'] == '0': return
    
    save_data(details, database)
    list_of_products.execute("UPDATE products SET done = 1 WHERE product = '%s'"%(product))
    list_of_products.commit()


In [ ]:
database = sqlite3.connect("data.db")
cur = database.cursor()
try:
    cur.execute("""CREATE TABLE laptops(
        product_id TEXT ,
        brand TEXT ,
        weight TEXT ,
        cpu TEXT,
        cpu_series TEXT,
        cpu_model TEXT,
        cpu_cache TEXT,
        freq TEXT,
        freq_bound TEXT,
        gpu TEXT,
        gpu_model TEXT,
        gpu_size TEXT,
        HDD TEXT,
        SSD TEXT,
        screen_size TEXT,
        screen_acc TEXT,
        port_vga TEXT,
        port_hdmi TEXT,
        port_thunderbolt TEXT,
        port_serial TEXT,
        usb3 TEXT,
        usb2 TEXT,
        usb4 TEXT,
        usbc TEXT,
        battery TEXT,
        os TEXT,
        ram TEXT,
        ram_kind TEXT, 
        price TEXT
    ) 
    """)
    database.commit()
    print('table created')
except:
    print('connected.')
list_of_products = sqlite3.connect("list.db")

products = list_of_products.execute("SELECT * FROM products").fetchall()
for product in products:
    extract_product_detail(product[0], database, list_of_products)